# Projet Impôt sur le Revenu - Modélisation et Simulation

## Objectif général

Ce projet modélise et simule le calcul de l'impôt sur le revenu français à deux niveaux :

1. **Modèle individuel** → calcul précis de l'impôt pour un foyer fiscal donné
2. **Modèle populationnel** → étude de l'impact d'une politique fiscale sur l'ensemble d'une population au fil du temps

**Barème utilisé :** 2024 - Source : service-public.gouv.fr

---


## 📚 Imports et Configuration


In [ ]:
# Imports nécessaires
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import odeint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports terminés")


## 🧮 Partie 1 - Modèle Individuel (Calculateur d'impôt)

### Classe CalculateurImpot


In [ ]:
class CalculateurImpot:
    """Calculateur d'impôt sur le revenu français - Modèle individuel"""
    
    def __init__(self, annee: int = 2024):
        self.annee = annee
        self.bareme_2024 = {
            'tranches': [0, 11497, 29315, 83823, 180294],
            'taux': [0, 0.11, 0.30, 0.41, 0.45]
        }
        
        # Paramètres pour la décote et le plafonnement
        self.decote_seuil_celibataire = 1726
        self.decote_seuil_couple = 2854
        self.plafond_quotient_familial = 1576
        
    def calculer_parts(self, nb_adultes: int, nb_enfants: int) -> float:
        """Calcule le nombre de parts du quotient familial"""
        if nb_adultes not in [1, 2]:
            raise ValueError("Le nombre d'adultes doit être 1 ou 2")
            
        # Parts de base
        if nb_adultes == 1:
            parts = 1.0
        else:  # nb_adultes == 2
            parts = 2.0
            
        # Parts supplémentaires pour les enfants
        parts += nb_enfants * 0.5
        
        return parts
    
    def calculer_impot_par_part(self, revenu_par_part: float) -> float:
        """Calcule l'impôt pour une part selon le barème progressif"""
        if revenu_par_part <= 0:
            return 0
            
        impot = 0
        tranches = self.bareme_2024['tranches']
        taux = self.bareme_2024['taux']
        
        for i in range(len(tranches) - 1):
            if revenu_par_part > tranches[i]:
                montant_imposable = min(revenu_par_part, tranches[i + 1]) - tranches[i]
                impot += montant_imposable * taux[i]
                
        # Dernière tranche (au-delà de 180294€)
        if revenu_par_part > tranches[-1]:
            montant_imposable = revenu_par_part - tranches[-1]
            impot += montant_imposable * taux[-1]
            
        return impot
    
    def appliquer_decote(self, impot_brut: float, nb_adultes: int) -> float:
        """Applique la décote si applicable"""
        if nb_adultes == 1:
            seuil = self.decote_seuil_celibataire
            decote = seuil - 0.75 * impot_brut
        else:  # nb_adultes == 2
            seuil = self.decote_seuil_couple
            decote = seuil - 0.75 * impot_brut
            
        if decote > 0:
            return max(0, impot_brut - decote)
        else:
            return impot_brut
    
    def calculer_impot(self, revenu: float, nb_adultes: int, nb_enfants: int) -> Dict:
        """Calcule l'impôt total pour un foyer fiscal"""
        # 1. Calcul du nombre de parts
        parts = self.calculer_parts(nb_adultes, nb_enfants)
        
        # 2. Revenu par part
        revenu_par_part = revenu / parts
        
        # 3. Impôt par part
        impot_par_part = self.calculer_impot_par_part(revenu_par_part)
        
        # 4. Impôt total avant décote
        impot_brut = parts * impot_par_part
        
        # 5. Application de la décote
        impot_final = self.appliquer_decote(impot_brut, nb_adultes)
        
        # Calcul des taux
        taux_marginal = self.calculer_taux_marginal(revenu_par_part)
        taux_moyen = impot_final / revenu if revenu > 0 else 0
        
        return {
            'revenu_total': revenu,
            'nb_adultes': nb_adultes,
            'nb_enfants': nb_enfants,
            'parts': parts,
            'revenu_par_part': revenu_par_part,
            'impot_par_part': impot_par_part,
            'impot_brut': impot_brut,
            'impot_final': impot_final,
            'taux_marginal': taux_marginal,
            'taux_moyen': taux_moyen
        }
    
    def calculer_taux_marginal(self, revenu_par_part: float) -> float:
        """Calcule le taux marginal d'imposition"""
        tranches = self.bareme_2024['tranches']
        taux = self.bareme_2024['taux']
        
        for i in range(len(tranches) - 1):
            if tranches[i] < revenu_par_part <= tranches[i + 1]:
                return taux[i]
                
        # Au-delà de la dernière tranche
        if revenu_par_part > tranches[-1]:
            return taux[-1]
            
        return taux[0]  # Tranche 0%

print("✅ Classe CalculateurImpot créée")
